In [1]:
# !pip install --upgrade tensorflow
!pip install tensorflow==2.15
# !pip install --upgrade tensorflow
!pip install -U tf_keras # Keras 2
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

!pip install transformers datasets tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 96.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 107.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 39.8 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      

In [2]:
!pip install datasets
!pip install transformers[torch]
!pip install evaluate
!pip install peft
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 8.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [3]:
# !pip install --upgrade transformers

In [4]:
# !pip show tensorflow
# !pip show Keras

In [5]:
import os
import tensorflow as tf
from transformers import TFElectraForSequenceClassification, ElectraTokenizer
from datasets import load_dataset, load_metric
from google.colab import drive

In [6]:
from datasets import load_dataset, load_metric
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
)
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
import evaluate
import torch
import numpy as np

In [7]:
torch.backends.cuda.matmul.allow_tf32 = False

In [8]:
drive.mount('/content/drive')

# Checkpoints and final model
checkpoint_dir = '/content/drive/My Drive/W266_Final_Project/final_project/LoRA_Finetune_BASE_GLUE/albert-base-qnli_finetune_allLayers_6epochs'
final_model_path = '/content/drive/My Drive/W266_Final_Project/final_project/LoRA_Finetune_BASE_GLUE/albert-base-qnli-final_finetune_allLayers_6epochs'


Mounted at /content/drive


In [9]:
# Load ELECTRA BASE model and tokenizer
model_checkpoint = 'albert-base-v2'
# tokenizer = ElectraTokenizer.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# model = TFElectraForSequenceClassification.from_pretrained(model_name, num_labels=2)

id2label = {
    0: "neg",
    1: "pos"
}

label2id = {
    "neg": 0,
    "pos": 1
}

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
for name, param in model.named_parameters():
  print(name, param.requires_grad)

albert.embeddings.word_embeddings.weight True
albert.embeddings.position_embeddings.weight True
albert.embeddings.token_type_embeddings.weight True
albert.embeddings.LayerNorm.weight True
albert.embeddings.LayerNorm.bias True
albert.encoder.embedding_hidden_mapping_in.weight True
albert.encoder.embedding_hidden_mapping_in.bias True
albert.encoder.albert_layer_groups.0.albert_layers.0.full_layer_layer_norm.weight True
albert.encoder.albert_layer_groups.0.albert_layers.0.full_layer_layer_norm.bias True
albert.encoder.albert_layer_groups.0.albert_layers.0.attention.query.weight True
albert.encoder.albert_layer_groups.0.albert_layers.0.attention.query.bias True
albert.encoder.albert_layer_groups.0.albert_layers.0.attention.key.weight True
albert.encoder.albert_layer_groups.0.albert_layers.0.attention.key.bias True
albert.encoder.albert_layer_groups.0.albert_layers.0.attention.value.weight True
albert.encoder.albert_layer_groups.0.albert_layers.0.attention.value.bias True
albert.encoder.alb

In [12]:
model

AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768,

In [13]:
_= model.to("cuda")

In [14]:
# Load and preprocess the GLUE QNLI dataset
# https://huggingface.co/docs/datasets/v1.1.2/loading_datasets.html
# https://huggingface.co/docs/datasets/en/process
# https://huggingface.co/datasets/nyu-mll/glue
dataset = load_dataset("glue", "qnli")
metric = load_metric("glue", "qnli")

encoded_dataset = dataset.map(lambda examples: tokenizer(examples["question"],
                                                         examples["sentence"],
                                                         truncation=True,
                                                         padding="max_length",
                                                         return_tensors="tf"),
                              batched=True)

Generating train split:   0%|          | 0/104743 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5463 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5463 [00:00<?, ? examples/s]

<ipython-input-14-86000d1cd3d5>:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "qnli")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Map:   0%|          | 0/104743 [00:00<?, ? examples/s]

Map:   0%|          | 0/5463 [00:00<?, ? examples/s]

Map:   0%|          | 0/5463 [00:00<?, ? examples/s]

In [21]:
# from datasets import list_metrics
# metric = load_metric('accuracy')

# def compute_metrics(p):
#     predictions, labels = p
#     predictions = np.argmax(predictions, axis=1)
#     return metric.compute(predictions=predictions, references=labels)

In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [23]:
lr = 5e-5
batch_size = 32
num_epochs = 6

In [24]:
training_args = TrainingArguments(
    output_dir=checkpoint_dir,
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=1,
    load_best_model_at_end=True,
)

In [25]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [26]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.557700,0.285979,0.883397,0.884286,0.883397,0.883364
2,0.092100,0.296315,0.882848,0.886918,0.882848,0.882613
3,0.014600,0.314386,0.877906,0.884258,0.877906,0.877496
4,0.014900,0.409586,0.894197,0.894257,0.894197,0.894200
5,0.001000,0.568513,0.892916,0.892934,0.892916,0.892918


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.557700,0.285979,0.883397,0.884286,0.883397,0.883364
2,0.092100,0.296315,0.882848,0.886918,0.882848,0.882613
3,0.014600,0.314386,0.877906,0.884258,0.877906,0.877496
4,0.014900,0.409586,0.894197,0.894257,0.894197,0.894200
5,0.001000,0.568513,0.892916,0.892934,0.892916,0.892918
6,0.000700,0.668291,0.891452,0.891557,0.891452,0.891454


TrainOutput(global_step=19644, training_loss=0.1685475633637059, metrics={'train_runtime': 16050.8723, 'train_samples_per_second': 39.154, 'train_steps_per_second': 1.224, 'total_flos': 1.501895323588608e+16, 'train_loss': 0.1685475633637059, 'epoch': 6.0})

In [27]:
trainer.evaluate()

{'eval_loss': 0.28597933053970337,
 'eval_accuracy': 0.8833974006955885,
 'eval_precision': 0.8842858918589614,
 'eval_recall': 0.8833974006955885,
 'eval_f1': 0.8833636544270601,
 'eval_runtime': 50.0504,
 'eval_samples_per_second': 109.15,
 'eval_steps_per_second': 3.417,
 'epoch': 6.0}

In [28]:
# Save the final model
trainer.save_model(final_model_path)

22.9 GB GPU RAM